# Use NCBI Datasets to Retrieve Information on Genes

The objective of this notebook is to use the `ncbi.datasets` python library to download and extract gene data.

In this example, we will get information about the Gonadotropin Releasing Hormone Receptor (GNRHR) gene family, which plays a key role in sexual development and function across vertebrates.

This notebook will demonstrate the following features that are available through NCBI Datasets for retrieving gene data:  
1. Getting gene metadata as a JSON-formatted gene descriptor, by specifying either an NCBI Gene ID or gene symbol  
2. Downloading gene datasets, which include gene, transcript, and protein sequences, and a data report (gene metadata in a more human-readable yaml format)

Gonadotropin Releasing Hormone (GnRH) binds and activates the GnRH receptor (GnRHR) to stimulate secretion of follicle stimulating hormone (FSH) and luteinizing hormones (LH). These hormones in turn regulate repoductive development and function. 

While the role of GnRH in reproduction is conserved across vertebrates, GnRH receptor copy number is variable. 
In humans and some primates, there are two GnRHR genes, while in fish and amphibians, three GnRHR genes have been identified, with additional duplications observed in some fish. Additional receptors and GnRH ligands suggests that new functions could have been acquired by the additional gene copies[1].

[1] Moncaut N, Somoza G, Power DM, Canário AV. Five gonadotrophin-releasing hormone receptors in a teleost fish: isolation, tissue distribution and phylogenetic relationships. J Mol Endocrinol. 2005 Jun;34(3):767-79. doi: 10.1677/jme.1.01757. PMID: 15956346.


First we will load the libraries necessary to run this notebook. This includes:
1. the ncbi.datasets library 
2. pandas, the python data analysis library and 
3. pprint, which allows "pretty-printing" of Python data structures

In [1]:
# load all libraries
import pandas as pd
import ncbi.datasets


## Get gene descriptors for three human GnRHR genes 
First we're going to get gene descriptors (metadata) for three human GnRHR genes, GNRHR, GNRHR2, and GNRHR2P1, by specifying the NCBI Gene IDs for these genes. 

Gene descriptors provide gene metadata in a machine-readable JSON format.

Gene descriptors contain a lot of interesting metadata and it's easy to pull out just the fields that you're interested in. As an example, here we'll show gene symbols, chromosome number and the corresponding SwissProt accession for the three genes.

In [1]:
# start a datasets gene API instance
api_client = ncbi.datasets.ApiClient()
ds_gene_instance = ncbi.datasets.GeneDatasetDescriptorsApi(api_client)

# Retrieve descriptors for three known gene-ids.
gene_descriptors = ds_gene_instance.gene_descriptors_by_id([2798, 114814, 404718])

# Look up the symbols for each gene
def report_on_gene_descriptors(gene_descriptors, leader='\t'):
    if gene_descriptors.errors:
        for error in gene_descriptors.errors:
            print(f'{leader}Error for: ({",".join(error.invalid_identifiers)})')

    if not gene_descriptors.genes:
        print(f'{leader}No descriptors found')
        return

    for gene in gene_descriptors.genes:
        print(f'{leader}{gene.gene_id} -> {gene.symbol} (Chromosome(s): {gene.chromosomes}, SwissProt: {gene.swiss_prot_accessions})')

report_on_gene_descriptors(gene_descriptors)

## Get gene descriptors for GNRHR genes throughout primates
Now we're going to get gene descriptors (metadata) for a large set of GnRHR genes in primates. To get these gene descriptors we can specify gene symbol and species-level Tax IDs.  
In order to get these species-level Tax ID, we're going to use the Datasets taxtree API to get the species-level TaxIDs under the taxonomic rank of primate.
We're going to search for two symbols, GNRHR and GNRHR2. Interestingly, very few primates have an annotated gene ortholog of human GNRHR2, and this is reflected in the results. In fact, the only non-human primates for which we can find GNRHR2 is in Rhesus monkey (Macaca mulatta) and white-tufted-ear marmoset.
For those primates without an apparent GNRHR2, it's also possible that we might miss the gene because of inconsistent (or missing) nomenclature.
In a future iteration of Datasets, we plan to give users access to orthology information that would allow retrieval of a more comprehensive set of gene homologs across organisms.

In [1]:
#Get gene descriptors by gene symbol + organism name for all primates
ds_tax_instance = ncbi.datasets.TaxTreeApi(api_client)
primate_tax_id = 9443

def species_tax_ids(tree):
    
    if tree.children:
        for child in tree.children:
            yield from species_tax_ids(child)
        
    if tree.rank == 'SPECIES':
        yield tree.tax_id, tree.sci_name, tree.common_name

primate_tax_tree = ds_tax_instance.tax_tree_by_tax_id(primate_tax_id)

symbols = ['GNRHR', 'GNRHR2']
for tax_id, sci_name, common_name in species_tax_ids(primate_tax_tree):
    print(f'Fetch for {common_name} ({sci_name})')
    gene_descriptors = ds_gene_instance.gene_descriptors_by_tax_and_symbol(symbol=symbols, tax_token=tax_id)
    report_on_gene_descriptors(gene_descriptors)


## Get gene descriptors for GNRHR genes across vertebrates and build a table

Let's expand the taxonomic scope even further, and look at a selection of vertebrates.

We'll use a pre-determined list of Gene IDs to get gene descriptors for these genes and build an easily readable table with key information about these genes.

In [1]:
# extract fields of interest from descriptors class to build a table
cols = '''
common_name
taxonomic_name
symbol
type
chromosome
num_transcripts
ensembl_id
omim_id
uniprot_id
nomenclature_id
nomenclature_auth
genome_coordinates
'''
cols = cols.split('\n')[1:-1]

def _range_repr(range):
    ret = []
    for interval in range:
        ret.append(f'{interval.begin}_{interval.end}')
    return ','.join(ret)

def _ranges_repr(ranges):
    ret = []
    for range in ranges:
        ret.append(f'{range.accession_version}:{_range_repr(range.range)}')
    return ','.join(ret)

# specify genes of interest and retrieve descriptors
gene_ids = [2798, 114814, 404718, 14715, 109324103, 109309182, 281798, 395368, 403718, 427517, 471226, 7226731, 100001586, 100135415, 100135416, 100135417, 100136028, 100270671, 100270672, 101318246, 101932446, 101935915, 101953943, 102193667, 102202954, 102205592, 102346610, 102363373, 102364206, 102366752, 102536567, 102687824, 102694185, 102770612, 103899900, 103899926, 105916404, 105919697, 105934126, 108392639, 109987527, 109994050, 109999298, 110488224, 110495632, 110496352, 110513414, 110520912, 112994411, 112996301, 114645297, 114667483]
gene_descriptors = ds_gene_instance.gene_descriptors_by_id(gene_ids)

# collect elements of the descriptor class into a dictionary based on each gene ID
table_data = {}
for gene in gene_descriptors.genes:
    table_data[gene.gene_id] = [gene.common_name]
    table_data[gene.gene_id].append(gene.taxname)
    table_data[gene.gene_id].append(gene.symbol)
    table_data[gene.gene_id].append(gene.type)
    table_data[gene.gene_id].append(gene.chromosome)
    if gene.transcripts:
        table_data[gene.gene_id].append(len(gene.transcripts))
    else:
        table_data[gene.gene_id].append(0)
    table_data[gene.gene_id].append(gene.ensembl_gene_ids)
    table_data[gene.gene_id].append(gene.omim_ids)
    table_data[gene.gene_id].append(gene.swiss_prot_accessions)
    if gene.nomenclature_authority:
        table_data[gene.gene_id].append(gene.nomenclature_authority.identifier)
        table_data[gene.gene_id].append(gene.nomenclature_authority.authority)
    else:
        table_data[gene.gene_id].append(None)
        table_data[gene.gene_id].append(None)        
    table_data[gene.gene_id].append(_ranges_repr(gene.genomic_ranges))

        
df = pd.DataFrame.from_dict(table_data, orient='index', columns=cols)
df.index.name = 'gene_id'
df

## Use gene descriptor data to build a simple table of gene copy numbers

As we mentioned before, while the role of the gonadotropin releasing hormone and receptor in reproduction is highly conserved across vertebrates, gene copy number for the receptor genes is not. Let's take a quick look at how gene count varies in different organisms.  
Note that in our list, you can see that rainbow trout has the most gene copies at 6, while numerous mammals, including mouse, dolphin and alpaca, only have a single annotated gene copy. 

In [1]:
# plot gene count based on organism
gene_cnt = df.groupby('common_name')['symbol'].count().reset_index()
gene_cnt.columns = ['organism', 'gene_count']
gene_cnt.sort_values('gene_count', ascending=False, inplace=True)
gene_cnt

## Use gene datasets to build a transcript-focused table

Finally, we are going to download a gene dataset for the human GnRH receptor genes, and use metadata included within the dataset to build a transcript-focused table.
We'll take this metadata from the data report file, which is similar to the gene descriptor described above, but in a more human-readable yaml format.  
Note that gene datasets include a data table that is gene-oriented, with one gene record per row. Here, we are building a custom transcript-oriented table.


In [1]:
%%time

ds_gene_download_instance = ncbi.datasets.DownloadApi(ncbi.datasets.ApiClient())
gene_ids = [2798, 114814, 404718]
gene_ds_download = ds_gene_download_instance.download_gene_package(gene_ids, _preload_content=False)

## write to a zip file 
zipfile_name = 'gene_ds.zip'
with open(zipfile_name, 'wb') as f:
    f.write(gene_ds_download.data)

print(f'Download complete to {zipfile_name}')

In [1]:
!unzip -v {zipfile_name}

In [1]:
import json
import zipfile

from google.protobuf.json_format import ParseDict
import yaml

import ncbi.datasets.v1alpha1.reports.gene_pb2 as gene_report_pb2

def gene_report_for(path_to_zipfile):
    '''
    Return an object representing the data report.
    path_to_zipfile: The relative path to the zipfile containing the virus data report
    '''
    with zipfile.ZipFile(path_to_zipfile, 'r') as zip:
        gene_report_as_dict = yaml.safe_load(zip.read('ncbi_dataset/data/data_report.yaml'))
    gene_report = gene_report_pb2.GeneDescriptors()
    ParseDict(gene_report_as_dict, gene_report)
    return gene_report

def _5prime_len(transcript):
    if not transcript.cds or not transcript.cds.range:
        return None
    return transcript.cds.range[0].begin - 1

def _3prime_len(transcript):
    if not transcript.cds or not transcript.cds.range:
        return None
    return transcript.length - transcript.cds.range[0].end

gene_report = gene_report_for(zipfile_name)

rows = []
for gene in gene_report.genes:

    # transcripts for each gene are embedded as lists and require additional handling
    for transcript in gene.transcripts:
        rows.append({
            'gene_id': gene.gene_id,
            'gene_symbol': gene.symbol,
            'gene_taxonomy': gene.taxname,            
            'accVer': transcript.accession_version,
            'name': transcript.name,
            'length': transcript.length,
            '5`UTR_len': _5prime_len(transcript),
            '3`UTR_len': _3prime_len(transcript),
            'protAccVer': transcript.protein.accession_version or None,
            'protName': transcript.protein.isoform_name or None,
            'protLength': transcript.protein.length or None,
            'exonAccVer': transcript.exons.accession_version,
            'numExons': len(transcript.exons.range),
        })

transcript_table = pd.DataFrame(rows)

transcript_table
